## Evaluating CF generation
1. Probability Increase
2. proximity
3. sparsity
4. Cohesive


In [26]:
import pandas as pd
import torch.nn as nn
import torch
import os
import numpy as np
import pickle



In [2]:
def load_data(data):
    return np.load(data)

class PimaClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden1 = nn.Linear(8, 32)
        self.act1 = nn.ReLU()
        self.hidden2 = nn.Linear(32, 64)
        self.act2 = nn.ReLU()
        self.hidden3 = nn.Linear(64, 16)
        self.act3 = nn.ReLU()
        self.output = nn.Linear(16, 1)
        self.act_output = nn.Sigmoid()

    def forward(self, x):
        x = self.act1(self.hidden1(x))
        x = self.act2(self.hidden2(x))
        x = self.act3(self.hidden3(x))
        x = self.act_output(self.output(x))
        return x
    
    
def load_model(input_folder = 'model_training/model'):
    model = torch.load(input_folder)
    model.eval()
    return model
    
def eval_model(model, input):
    # print(input)
    with torch.no_grad():
        prob = model(input)
    return prob.tolist()[0]

In [3]:
model = load_model(input_folder = './../model_training/model')

### Evaluate average probability shift

In [27]:
def evaluate_model_for_df(df , model):
    probability_list = []
    if df.shape[0] == 0:
        return 0
    else:
        for index, row in df.iterrows():
            x = row.to_numpy()
            prob = eval_model(model, torch.tensor(x, dtype=torch.float32))
            probability_list.append(prob)
        return sum(probability_list)/len(probability_list)
        

def average_probability_shift(input_data_loc, dice_loc, ge_cf_loc, ge_nsga2_loc, certif_cf_loc, model):
    metric_dict = {'initial_prob' : [],
                   'dice_prob' : [],
                   'Our_method_prob' : [],
                   'certif_prob' : [],
                   'GE_NSGAII_prob': []}
        
    for i in os.listdir(input_data_loc):
        if i=='.DS_Store':
            continue
        # print(i)
        input_array_location = input_data_loc + '/' + i + '/' + 'input_data.npy'
        input = load_data(input_array_location)
        prob = eval_model(model,  torch.tensor(input, dtype=torch.float32))
        # print(prob)
        dice_df = pd.read_csv(dice_loc +  i + '.csv')
        dice_df.drop(columns=['outcome'], inplace=True)
        Our_method_df  = pd.read_csv(ge_cf_loc +  i + '.csv')
        certif_df = pd.read_csv(certif_cf_loc +  i + '.csv')
        NSGAII_df = pd.read_csv(ge_nsga2_loc +  i + '.csv')
        
        metric_dict['initial_prob'].append(prob)
        metric_dict['dice_prob'].append(evaluate_model_for_df(dice_df, model))
        metric_dict['Our_method_prob'].append(evaluate_model_for_df(Our_method_df, model))
        metric_dict['certif_prob'].append(evaluate_model_for_df(certif_df, model))
        metric_dict['GE_NSGAII_prob'].append(evaluate_model_for_df(NSGAII_df, model))
        
    return metric_dict

def save_dict(d, filename):
    with open(filename, 'wb') as handle:
        pickle.dump(d, handle, protocol=pickle.HIGHEST_PROTOCOL)
        

In [28]:
input_data_loc = './../output/NSGAIII/'
dice_loc = 'Dice_cf/'
ge_cf_loc = 'Ge_cf/'
ge_nsga2_loc = 'Ge_NSGAII_cf/'
certif_cf_loc = 'Certif_cf/'

metric = average_probability_shift(input_data_loc, dice_loc, ge_cf_loc, ge_nsga2_loc, certif_cf_loc, model)

In [29]:
def average_shift(metric):
    metric_dict = {}
    dice_shift = []
    our_method_shift = []
    certif_shift = []
    nsga2_shift = []
    for i in range(0, 30):
        if metric['dice_prob'][i]!= 0:
            dice_shift.append(metric['dice_prob'][i] - metric['initial_prob'][i])
        else:
            dice_shift.append(0)
        
        if metric['Our_method_prob'][i]!= 0:
            our_method_shift.append(metric['Our_method_prob'][i] - metric['initial_prob'][i])
        else:
            our_method_shift.append(0)
        
        if metric['certif_prob'][i]!= 0:
            certif_shift.append(metric['certif_prob'][i] - metric['initial_prob'][i])
        else:
            certif_shift.append(0)
        
        if metric['GE_NSGAII_prob'][i]!= 0:
            nsga2_shift.append(metric['GE_NSGAII_prob'][i] - metric['initial_prob'][i])
        else:
            nsga2_shift.append(0)
        
    
    metric_dict['Dice'] =     sum(dice_shift)/len(dice_shift)
    metric_dict['Certif'] = sum(certif_shift)/len(certif_shift)
    metric_dict['Our_method'] = sum(our_method_shift)/len(our_method_shift)
    metric_dict['GE_NSGAII'] = sum(nsga2_shift)/len(nsga2_shift)
    
    d = {'Dice': dice_shift, 'Certif': certif_shift,
                              'Our_method': our_method_shift, 'GE_NSGAII':nsga2_shift}
    

    
    return metric_dict, d
        
        
    

In [30]:
output, daf = average_shift(metric)
save_dict(daf, 'data_statistical_test/average_shift.pickle')

## Proximity comparison : average distance of counterfactuals from input 
- lower the better

In [20]:
from sklearn.metrics import DistanceMetric

def mean(l):
    return sum(l)/len(l)

def calculate_l2_distance_sklearn(array, df):
    
    # Create a DistanceMetric object for Euclidean distance
    if df.shape[0] == 0:
        print("no counterfactuals")
        return []
    
    euclidean = DistanceMetric.get_metric('euclidean')
    
    # Ensure array is 2D for pairwise
    array_2d = array.reshape(1, -1)
    
    # Calculate distances using the pairwise method
    distances = euclidean.pairwise(array_2d, df.values)
    
    # Convert distances to a list
    distances_list = distances.flatten().tolist()
    
    return distances_list


def calculate_proximity(input_data_loc, dice_loc, ge_cf_loc, ge_nsga2_loc, certif_cf_loc, model):
    metric_dict = {'Dice' : [],
                   'Our_method' : [],
                   'Certif' : [],
                   'GE_NSGAII': []}
        
    for i in os.listdir(input_data_loc):
        if i=='.DS_Store':
            continue
        # print(i)
        input_array_location = input_data_loc + '/' + i + '/' + 'input_data.npy'
        input = load_data(input_array_location)
        # print(i)
        # print(prob)
        dice_df = pd.read_csv(dice_loc +  i + '.csv')
        dice_df.drop(columns=['outcome'], inplace=True)
        Our_method_df  = pd.read_csv(ge_cf_loc +  i + '.csv')
        certif_df = pd.read_csv(certif_cf_loc +  i + '.csv')
        NSGAII_df = pd.read_csv(ge_nsga2_loc +  i + '.csv')
        
        if dice_df.shape[0] > 0:
            metric_dict['Dice'].append(mean(calculate_l2_distance_sklearn(input, dice_df)))
        if Our_method_df.shape[0] > 0:
            metric_dict['Our_method'].append(mean(calculate_l2_distance_sklearn(input, Our_method_df)))
        if certif_df.shape[0] > 0:
            metric_dict['Certif'].append(mean(calculate_l2_distance_sklearn(input, certif_df)))
        if NSGAII_df.shape[0] > 0:
            metric_dict['GE_NSGAII'].append(mean(calculate_l2_distance_sklearn(input, NSGAII_df)))
    
    d = {'Dice': metric_dict['Dice'], 'Certif': metric_dict['Certif'],
                              'Our_method': metric_dict['Our_method'], 'GE_NSGAII':metric_dict['GE_NSGAII']}
    
    # print(d)
    out_metric = {}
    for key, value in metric_dict.items():
        out_metric[key] = mean(value)
    return out_metric, d
        

In [31]:
proximity, d_proximity = calculate_proximity(input_data_loc, dice_loc, ge_cf_loc, ge_nsga2_loc, certif_cf_loc, model)

save_dict(d_proximity, 'data_statistical_test/proximity.pickle')

proximity

{'Dice': 375.8579123404342,
 'Our_method': 58.54761684658484,
 'Certif': 60.09701095190148,
 'GE_NSGAII': 56.535571946582266}

## Sparsity: Average feature change in inputs
- lower the better

In [22]:
def calculate_feature_changes(array, df):
    changes = []
    for index, row in df.iterrows():
        # Compare array with row and count the number of differences
        # print('row {} and array {}'.format(row.values, array))
        num_changes = np.sum(array != row.values)
        # print('num_changes: {}'.format(num_changes))
        changes.append(num_changes)
    return changes


def calculate_sparsity(input_data_loc, dice_loc, ge_cf_loc, ge_nsga2_loc, certif_cf_loc, model):
    metric_dict = {'Dice' : [],
                   'Our_method' : [],
                   'Certif' : [],
                   'GE_NSGAII': []}
        
    for i in os.listdir(input_data_loc):
        if i=='.DS_Store':
            continue
        # print(i)
        input_array_location = input_data_loc + '/' + i + '/' + 'input_data.npy'
        input = load_data(input_array_location)
        # print(i)
        # print(prob)
        dice_df = pd.read_csv(dice_loc +  i + '.csv')
        dice_df.drop(columns=['outcome'], inplace=True)
        Our_method_df  = pd.read_csv(ge_cf_loc +  i + '.csv')
        certif_df = pd.read_csv(certif_cf_loc +  i + '.csv')
        NSGAII_df = pd.read_csv(ge_nsga2_loc +  i + '.csv')
        
        if dice_df.shape[0] > 0:
            metric_dict['Dice'].append(mean(calculate_feature_changes(input, dice_df)))
        if Our_method_df.shape[0] > 0:
            metric_dict['Our_method'].append(mean(calculate_feature_changes(input, Our_method_df)))
        if certif_df.shape[0] > 0:
            metric_dict['Certif'].append(mean(calculate_feature_changes(input, certif_df)))
        if NSGAII_df.shape[0] > 0:
            metric_dict['GE_NSGAII'].append(mean(calculate_feature_changes(input, NSGAII_df)))
    
    d = {'Dice': metric_dict['Dice'], 'Certif': metric_dict['Certif'],
                              'Our_method': metric_dict['Our_method'], 'GE_NSGAII':metric_dict['GE_NSGAII']}
    
    # print(d)
    
    
    out_metric = {}
    for key, value in metric_dict.items():
        out_metric[key] = mean(value)
    return out_metric, d

In [32]:
sparsity, d_sparsity = calculate_sparsity(input_data_loc, dice_loc, ge_cf_loc, ge_nsga2_loc, certif_cf_loc, model)

save_dict(d_sparsity, 'data_statistical_test/sparsity.pickle')

sparsity

{'Dice': 1.7296210155694045,
 'Our_method': 3.41060951585735,
 'Certif': 7.996666666666667,
 'GE_NSGAII': 3.5505844448873463}

## Cohesive : how close these individuals are with each other
1. normalized Average distance between all the individual with the centroid of data 
2. higher the better

In [34]:
def distance_between_all_counterfactuals(df):
    if not df.applymap(lambda x: isinstance(x, (int, float))).all().all():
        raise ValueError("DataFrame contains non-numeric data.")
    
    data = df.values
    euclidean = DistanceMetric.get_metric('euclidean')
    dist_matrix = euclidean.pairwise(data)
    
    distances = []
    n = len(dist_matrix)
    for i in range(n):
        for j in range(i + 1, n):
            distances.append(dist_matrix[i, j])
    
    return distances

def calculate_cohesive(input_data_loc, dice_loc, ge_cf_loc, ge_nsga2_loc, certif_cf_loc, model):
    metric_dict = {'Dice' : [],
                   'Our_method' : [],
                   'Certif' : [],
                   'GE_NSGAII': []}
        
    for i in os.listdir(input_data_loc):
        if i=='.DS_Store':
            continue
        # print(i)
        input_array_location = input_data_loc + '/' + i + '/' + 'input_data.npy'
        input = load_data(input_array_location)
        # print(i)
        # print(prob)
        dice_df = pd.read_csv(dice_loc +  i + '.csv')
        dice_df.drop(columns=['outcome'], inplace=True)
        Our_method_df  = pd.read_csv(ge_cf_loc +  i + '.csv')
        certif_df = pd.read_csv(certif_cf_loc +  i + '.csv')
        NSGAII_df = pd.read_csv(ge_nsga2_loc +  i + '.csv')
    
        if dice_df.shape[0] > 1:
            metric_dict['Dice'].append(mean(distance_between_all_counterfactuals(dice_df)))
        if Our_method_df.shape[0] > 1:
            metric_dict['Our_method'].append(mean(distance_between_all_counterfactuals(Our_method_df)))
        if certif_df.shape[0] > 1:
            metric_dict['Certif'].append(mean(distance_between_all_counterfactuals(certif_df)))
        if NSGAII_df.shape[0] > 1:
            metric_dict['GE_NSGAII'].append(mean(distance_between_all_counterfactuals(NSGAII_df)))
    
    d = {'Dice': metric_dict['Dice'], 'Certif': metric_dict['Certif'],
                              'Our_method': metric_dict['Our_method'], 'GE_NSGAII':metric_dict['GE_NSGAII']}
    
 
    
    out_metric = {}
    for key, value in metric_dict.items():
        out_metric[key] = 1/mean(value)
    return out_metric, d

In [35]:
cohesive, d_cohesive = calculate_cohesive(input_data_loc, dice_loc, ge_cf_loc, ge_nsga2_loc, certif_cf_loc, model)

save_dict(d_cohesive, 'data_statistical_test/cohesive.pickle')


In [36]:
from scipy.spatial.distance import mahalanobis
from sklearn.preprocessing import normalize

def normalize_dict(data_dict):
    values = np.array(list(data_dict.values())).reshape(1, -1)
    normalized_values = normalize(values, norm='l1')[0]
    normalized_dict = {key: normalized_values[i] for i, key in enumerate(data_dict.keys())}
    return normalized_dict

def find_mu_sigma(input_data_loc, output_column_name):
    data = pd.read_csv(input_data_loc)
    data.drop([output_column_name, 'Unnamed: 0'], axis=1, inplace=True)
    mu = np.mean(data.values, axis=0)
    sigma = np.cov(data.values.T)
    return mu, sigma


def mean_mahabolis_distances(df, mu, sigma):
    dist= []
    for index, row in df.iterrows():
        x = row.to_numpy()
        d = mahalanobis(x, mu, np.linalg.inv(sigma))
        dist.append(d)
    return mean(dist)
        
            


def closer_to_data_manifold(input_data_loc, dice_loc, ge_cf_loc, ge_nsga2_loc, certif_cf_loc, output_column_name, input_file_name):
    metric_dict = {'Dice' : [],
                   'Our_method' : [],
                   'Certif' : [],
                   'GE_NSGAII': []}
    
    mu, sigma = find_mu_sigma(input_file_name, output_column_name)
        
    for i in os.listdir(input_data_loc):
        if i=='.DS_Store':
            continue
        # print(i)
        input_array_location = input_data_loc + '/' + i + '/' + 'input_data.npy'
        input = load_data(input_array_location)
        dice_df = pd.read_csv(dice_loc +  i + '.csv')
        dice_df.drop(columns=['outcome'], inplace=True)
        Our_method_df  = pd.read_csv(ge_cf_loc +  i + '.csv')
        certif_df = pd.read_csv(certif_cf_loc +  i + '.csv')
        NSGAII_df = pd.read_csv(ge_nsga2_loc +  i + '.csv')
    
        if dice_df.shape[0] > 1:
            metric_dict['Dice'].append(mean_mahabolis_distances(dice_df, mu, sigma))
        if Our_method_df.shape[0] > 1:
            metric_dict['Our_method'].append(mean_mahabolis_distances(Our_method_df, mu, sigma))
        if certif_df.shape[0] > 1:
            metric_dict['Certif'].append(mean_mahabolis_distances(certif_df, mu, sigma))
        if NSGAII_df.shape[0] > 1:
            metric_dict['GE_NSGAII'].append(mean_mahabolis_distances(NSGAII_df, mu, sigma))
    
    
    d = {'Dice': metric_dict['Dice'], 'Certif': metric_dict['Certif'],
                              'Our_method': metric_dict['Our_method'], 'GE_NSGAII':metric_dict['GE_NSGAII']}
    
    
    out_metric = {}
    for key, value in metric_dict.items():
        out_metric[key] = 1/mean(value)
    
    out_metric = normalize_dict(out_metric)
    
    return out_metric, d

input_file_name = './../model_training/test.csv'
output_column_name = 'Outcome'

conformity_dict, d_conformity = closer_to_data_manifold(input_data_loc, dice_loc, ge_cf_loc, ge_nsga2_loc, certif_cf_loc, output_column_name, input_file_name)
print(conformity_dict)

save_dict(d_conformity, 'data_statistical_test/conformity.pickle')

{'Dice': 0.1761597439467549, 'Our_method': 0.3404403582388863, 'Certif': 0.16063577411069954, 'GE_NSGAII': 0.3227641237036593}


## goodness of counterfactuals 
goodness = (avg_probability_increase * cohesiveness * Conformity)/(proximity * sparsity)
- avg_probability_increase : increase the average probability of class
- cohesiveness : generated counterfactuals should be close to each other
- proximity: close to the input space
- sparsity: minimum feature change
- conformity : closeness to data manifold 
- higher is better

In [34]:



def goodness_of_cf(avg_prob_dict, cohesive_dict,  proximity_dict, sparsity_dict, conformity_dict):
    goodness_of_cf_dict = {}
    for key, value in avg_prob_dict.items():
        goodness_of_cf_dict[key] = (avg_prob_dict[key] * cohesive_dict[key] * conformity_dict[key])/(proximity_dict[key] * sparsity_dict[key])
    return normalize_dict(goodness_of_cf_dict)

goodness_of_cf = goodness_of_cf(output, cohesive, proximity, sparsity, conformity_dict)
goodness_of_cf

{'Dice': 0.008009241419037116,
 'Certif': 0.0486691905437965,
 'Our_method': 0.48953090592924686,
 'GE_NSGAII': 0.45379066210791935}